In [4]:
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import statsmodels.api as sm
import numpy as np
from statsmodels.stats.outliers_influence import variance_inflation_factor
from scipy import stats
from scipy.stats import shapiro
from scipy.stats.mstats import winsorize
from sklearn.linear_model import Lasso, Ridge
from sklearn.linear_model import ElasticNet
from sklearn.model_selection import train_test_split, cross_val_score
plt.rcParams['font.family'] = 'Malgun Gothic'
plt.rcParams['axes.unicode_minus'] = False

In [5]:
df = pd.read_csv('./dataset/label지정.csv', encoding='CP949')                        ## 성휘 파일 경로
# df = pd.read_excel('../eeez/전처리파일/label지정.xlsx', index_col=0)                ## 재인 파일 경로
# df = pd.read_csv('../데이터자료/label지정.csv', encoding='CP949')                   ## 의영 파일 경로
# df = pd.read_excel('./csv/dateset2_label최종.xlsx', index_col=0)                   ## 경록 파일 경로
df

,회사명,거래소코드,회계년도,상장일,상장폐지일,산업명,소속코드,[제조]매출액증가율,[제조]유형자산증가율,[제조]비유동자산증가율,...,[제조]기계투자효율,[제조]부가가치율,[제조]노동소득분배율,[제조]자본분배율,[제조]이윤분배율,[제조]유형자산회전율,[제조]이자보상배율(이자비용),year,month,label
0,(주)CMG제약,58820,2011-12-01,2001-08-31,NaN,의료용 물질 및 의약품 제조업,5,56.40,-5.90,-12.89,...,128.51,19.56,89.82,10.18,-43.44,1.77,3.53,2011,12,0
1,(주)CMG제약,58820,2012-12-01,2001-08-31,NaN,의료용 물질 및 의약품 제조업,5,-8.43,-4.12,-16.43,...,10.67,1.66,1749.87,-1649.87,-2473.07,1.71,-7.75,2012,12,0
2,(주)CMG제약,58820,2013-12-01,2001-08-31,NaN,의료용 물질 및 의약품 제조업,5,25.51,16.09,77.05,...,289.40,34.29,71.28,28.72,4.23,2.03,0.70,2013,12,0
3,(주)CMG제약,58820,2014-12-01,2001-08-31,NaN,의료용 물질 및 의약품 제조업,5,11.52,74.30,68.15,...,132.06,31.04,79.30,20.70,2.24,1.54,3.60,2014,12,0
4,(주)CMG제약,58820,2015-12-01,2001-08-31,NaN,의료용 물질 및 의약품 제조업,5,18.85,-5.37,1.66,...,90.92,15.96,144.10,-44.10,-84.58,1.48,-32.22,2015,12,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10657,흥구석유(주),24060,2016-12-01,1994-12-07,NaN,도매 및 상품 중개업,5,-7.26,-5.61,0.11,...,3684.02,4.86,48.08,51.92,31.14,3.67,7.66,2016,12,0
10658,흥구석유(주),24060,2017-12-01,1994-12-07,NaN,도매 및 상품 중개업,5,8.12,-0.56,-0.48,...,4786.26,4.39,47.63,52.37,30.24,4.10,5.91,2017,12,0
10659,흥구석유(주),24060,2018-12-01,1994-12-07,NaN,도매 및 상품 중개업,5,6.87,-0.36,-2.09,...,3064.10,4.44,47.47,52.53,32.14,4.40,12.66,2018,12,0
10660,흥구석유(주),24060,2019-12-01,1994-12-07,NaN,도매 및 상품 중개업,5,-9.35,-7.00,-2.63,...,3489.21,10.00,22.19,77.81,55.02,4.14,252.02,2019,12,0


In [6]:
df.describe()

,거래소코드,소속코드,[제조]매출액증가율,[제조]유형자산증가율,[제조]비유동자산증가율,[제조]총자본증가율,[제조]자기자본증가율,[제조]유동자산증가율,[제조]순이익증가율,[제조]재고자산증가율,...,[제조]기계투자효율,[제조]부가가치율,[제조]노동소득분배율,[제조]자본분배율,[제조]이윤분배율,[제조]유형자산회전율,[제조]이자보상배율(이자비용),year,month,label
count,10662.000000,10662.000000,10662.000000,10662.000000,10662.000000,10662.000000,10662.000000,10662.000000,1.066200e+04,10662.000000,...,1.066200e+04,10662.000000,10662.000000,10662.000000,10662.000000,10662.000000,1.066200e+04,10662.000000,10662.0,10662.000000
mean,92074.287188,4.993341,28.056523,84.564483,21.639397,13.746344,22.626214,17.352542,6.212778e+02,87.858023,...,1.485140e+06,10.562375,111.876595,-23.525442,-73.331389,14.732072,1.894793e+05,2015.869818,12.0,0.015757
std,106083.269592,0.354335,1454.454305,3093.493880,129.847723,51.742527,354.943960,82.628720,6.254670e+04,3275.800579,...,5.170670e+07,224.658324,1419.551265,1418.995549,2107.927519,102.169681,1.369595e+07,2.882518,0.0,0.124539
min,250.000000,4.000000,-100.000000,-100.000000,-99.470000,-95.840000,-1068.120000,-98.810000,-1.522079e+05,-99.970000,...,-2.657909e+09,-13958.620000,0.000000,-116701.060000,-190075.340000,0.000000,-4.493653e+06,2011.000000,12.0,0.000000
25%,39240.000000,5.000000,-9.657500,-5.257500,-4.167500,-3.217500,-1.920000,-8.830000,-3.159750e+01,-11.220000,...,7.957000e+01,12.020000,38.280000,2.385000,-0.297500,1.660000,-6.875000e-01,2013.000000,12.0,0.000000
50%,65950.000000,5.000000,4.300000,1.440000,4.485000,5.520000,5.870000,5.535000,0.000000e+00,0.000000,...,2.685100e+02,21.785000,57.065000,35.020000,10.985000,3.030000,2.470000e+00,2016.000000,12.0,0.000000
75%,105330.000000,5.000000,19.100000,17.970000,21.067500,17.827500,17.110000,23.542500,3.889500e+01,22.117500,...,1.303918e+03,34.447500,76.037500,52.097500,28.827500,6.407500,1.707750e+01,2018.000000,12.0,0.000000
max,950200.000000,6.000000,149805.530000,295498.650000,7919.440000,2579.670000,34640.710000,4274.050000,6.452079e+06,313488.540000,...,1.418999e+09,2377.220000,116801.060000,99.180000,3373.680000,6295.580000,1.000000e+09,2020.000000,12.0,1.000000


## Winsorizing

In [8]:
df_1 = df.copy()
df_1.drop(['회사명', '거래소코드', '회계년도', '상장일', '상장폐지일', '산업명', '소속코드','year', 'month'],axis=1,inplace=True)
for col in df_1.columns:
    df_1[col] = winsorize(df_1[col], limits= 0.01)

# Ridge

In [9]:

feature = df_1.drop(columns='label')
target = df_1['label']

#릿지 클래서
ridge = Ridge(alpha = 10)

neg_mse_scores = cross_val_score(ridge, feature, target, scoring="neg_mean_squared_error", cv=5)
rmse_scores = np.sqrt(-1 * neg_mse_scores)
avg_rmse = np.mean(rmse_scores)
print(neg_mse_scores,rmse_scores,avg_rmse)

ridge_alphas =[1000,2000,3000,4000,5000]
for alpha in ridge_alphas :
    ridge = Ridge(alpha = alpha)
    neg_mse_scores = cross_val_score(ridge, feature, target, scoring="neg_mean_squared_error", cv=5)
    avg_rmse = np.mean(np.sqrt(-1*neg_mse_scores))
    print('alpha {0} 일 떄 5folds 의 평균 RMSE : {1:.10f}' .format(alpha,avg_rmse))

# fig, axs = plt.subplots(figsize = (18,6), nrows = 1, ncols=5)

coeff_df = pd.DataFrame()

for pos, alpha in enumerate(ridge_alphas):
    ridge = Ridge(alpha= alpha)
    ridge.fit(feature,target)
    coeff = pd.Series(data=ridge.coef_, index=feature.columns)
    colname = 'alpha:' + str(alpha)
    coeff_df[colname] = coeff

    coeff = coeff.sort_values(ascending=False)
#     axs[pos].set_title(colname)
#     axs[pos].set_xlim(-3,6)
#     sns.barplot(x=coeff.values, y=coeff.index, ax =axs[pos])

# plt.show()


[-0.01193345 -0.01759934 -0.01315546 -0.01699339 -0.01257198] [0.10924032 0.13266249 0.11469726 0.13035871 0.11212483] 0.11981672159399459
alpha 1000 일 떄 5folds 의 평균 RMSE : 0.1198683457
alpha 2000 일 떄 5folds 의 평균 RMSE : 0.1199116651
alpha 3000 일 떄 5folds 의 평균 RMSE : 0.1199349066
alpha 4000 일 떄 5folds 의 평균 RMSE : 0.1199501318
alpha 5000 일 떄 5folds 의 평균 RMSE : 0.1199610267


## Alpha 값 조정하면서 최적의 Alpha값 찾기

In [10]:
ridge_alphas =[1000,2000,3000,4000,5000]
sort_column = 'alpha:'+str(ridge_alphas[0])
coeff_df.sort_values(by=sort_column, ascending=False)

,alpha:1000,alpha:2000,alpha:3000,alpha:4000,alpha:5000
[제조]순운전자본회전률,8.877626e-03,5.031828e-03,3.431583e-03,2.557495e-03,2.010174e-03
[제조]총자본회전률,5.669016e-03,2.929316e-03,1.858900e-03,1.302565e-03,9.684183e-04
[제조]비유동자산회전률,2.150611e-03,2.015404e-03,1.837735e-03,1.672157e-03,1.526047e-03
[제조]운전자본회전률,1.116456e-03,8.881443e-04,7.327282e-04,6.223918e-04,5.403157e-04
[제조]유동자산구성비율,9.767115e-04,9.109473e-04,8.749342e-04,8.519578e-04,8.358572e-04
[제조]매출채권회전률,7.780714e-04,7.763455e-04,7.749263e-04,7.732301e-04,7.713089e-04
[제조]총자본사업이익률,5.048783e-04,5.267562e-04,5.354428e-04,5.391369e-04,5.403650e-04
[제조]자기자본구성비율,4.259166e-04,4.453057e-04,4.523346e-04,4.551535e-04,4.560255e-04
[제조]1회전기간,2.766966e-04,1.431070e-04,9.679279e-05,7.314620e-05,5.879927e-05
[제조]매입채무회전기간,1.600568e-04,1.609018e-04,1.616238e-04,1.622181e-04,1.627204e-04


# Lasso

In [11]:
lasso = Lasso(alpha = 10)
lasso_alphas =[0,0.1,1,10,100]

neg_mse_scores = cross_val_score(lasso, feature, target, scoring="neg_mean_squared_error", cv=5)
rmse_scores = np.sqrt(-1 * neg_mse_scores)
avg_rmse = np.mean(rmse_scores)
print(neg_mse_scores,rmse_scores,avg_rmse)

# lasso_alphas =[100,10000,10000]
for alpha in lasso_alphas :
    lasso = Lasso(alpha = alpha)
    neg_mse_scores = cross_val_score(lasso, feature, target, scoring="neg_mean_squared_error", cv=5)
    avg_rmse = np.mean(np.sqrt(-1*neg_mse_scores))
    print('alpha {0} 일 떄 5folds 의 평균 RMSE : {1:10f}' .format(alpha,avg_rmse))

# fig, axs = plt.subplots(figsize = (18,6), nrows = 1, ncols=5)

coeff_df = pd.DataFrame()

for pos, alpha in enumerate(lasso_alphas):
    lasso = Lasso(alpha = alpha)
    lasso.fit(feature,target)
    coeff = pd.Series(data=lasso.coef_, index=feature.columns)
    colname = 'alpha:' + str(alpha)
    coeff_df[colname] = coeff

    coeff = coeff.sort_values(ascending=False)

[-0.01241827 -0.01914632 -0.0142275  -0.01783509 -0.01339871] [0.11143729 0.13837023 0.11927907 0.13354808 0.11575279] 0.12367749091851843


c:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:593: UserWarning: With alpha=0, this algorithm does not converge well. You are advised to use the LinearRegression estimator
  estimator.fit(X_train, y_train, **fit_params)
c:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: UserWarning: Coordinate descent with no regularization may lead to unexpected results and is discouraged.
  model = cd_fast.enet_coordinate_descent(
c:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 62.36248837631895, tolerance: 0.013866901160745708
  model = cd_fast.enet_coordinate_descent(
c:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:593: UserWarning: With alpha=0, this algorithm does not converge well. You are advised to use the LinearRegression estimat

alpha 0 일 떄 5folds 의 평균 RMSE :   0.119865
alpha 0.1 일 떄 5folds 의 평균 RMSE :   0.120284
alpha 1 일 떄 5folds 의 평균 RMSE :   0.122582
alpha 10 일 떄 5folds 의 평균 RMSE :   0.123677
alpha 100 일 떄 5folds 의 평균 RMSE :   0.123690


<ipython-input-11-37e426e0f024>:22: UserWarning: With alpha=0, this algorithm does not converge well. You are advised to use the LinearRegression estimator
  lasso.fit(feature,target)
c:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: UserWarning: Coordinate descent with no regularization may lead to unexpected results and is discouraged.
  model = cd_fast.enet_coordinate_descent(
c:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 74.84477915003872, tolerance: 0.016535284186831766
  model = cd_fast.enet_coordinate_descent(


## Alpha 값 조정하면서 최적의 Alpha값 찾기

In [12]:
lasso_alphas =[0,0.1,1,10,100]

sort_column = 'alpha:'+str(lasso_alphas[0])
coeff_df.sort_values(by=sort_column, ascending=False)


,alpha:0,alpha:0.1,alpha:1,alpha:10,alpha:100
[제조]순운전자본회전률,3.608254e-02,-0.000000e+00,-0.000000e+00,-0.000000e+00,-0.000000e+00
[제조]1회전기간,3.163814e-02,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
[제조]총자본회전률,2.370192e-02,-0.000000e+00,-0.000000e+00,-0.000000e+00,-0.000000e+00
[제조]비유동자산회전률,1.501526e-03,-0.000000e+00,-0.000000e+00,-0.000000e+00,-0.000000e+00
[제조]운전자본회전률,1.308476e-03,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
[제조]유동자산구성비율,1.172111e-03,0.000000e+00,-0.000000e+00,-0.000000e+00,-0.000000e+00
[제조]매출채권회전률,7.688042e-04,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
[제조]총자본사업이익률,4.036064e-04,-0.000000e+00,-0.000000e+00,-0.000000e+00,-0.000000e+00
[제조]자기자본구성비율,2.995827e-04,-0.000000e+00,-0.000000e+00,-0.000000e+00,-0.000000e+00
[제조]매입채무회전기간,1.576282e-04,7.090729e-05,0.000000e+00,0.000000e+00,0.000000e+00


In [13]:

# a = coeff_df[coeff_df["alpha:0.25"]<-0.0000000001] 
# b = coeff_df[coeff_df["alpha:0.25"]>0.00000001]

# a.info()

In [14]:
# feature = pd.concat([a,b])
# feature = feature["alpha:0.25"]
# feature = pd.DataFrame(feature)
# feature

# ElasticNet

In [15]:
from sklearn.linear_model import ElasticNet
elasticNet = ElasticNet(alpha = 10)
alphas =[0.07,0.1,0.5,1,3]


# fig, axs = plt.subplots(figsize = (18,6), nrows = 1, ncols=5)

coeff_df = pd.DataFrame()

for pos, alpha in enumerate(alphas):
    
    elasticNet = ElasticNet(alpha = alpha, l1_ratio=0.07)
    elasticNet.fit(feature,target)
    coeff = pd.Series(data=elasticNet.coef_, index=feature.columns)
    colname = 'alpha:' + str(alpha)
    coeff_df[colname] = coeff

    coeff = coeff.sort_values(ascending=False)

## Alpha 값 조정하면서 최적의 Alpha값 찾기

In [49]:
elasticNet_alphas =[0,0.1,1,10,100]

sort_column = 'alpha:'+str(elasticNet_alphas[0])
coeff_df.sort_values(by=sort_column, ascending=False)


,alpha:0,alpha:0.1,alpha:1,alpha:10,alpha:100
[제조]총자본회전률,3.144116e-02,-0.000000e+00,-0.000000e+00,-0.000000e+00,-0.000000e+00
[제조]순운전자본회전률,2.115825e-02,-0.000000e+00,-0.000000e+00,-0.000000e+00,-0.000000e+00
[제조]1회전기간,1.614939e-02,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
[제조]운전자본회전률,1.999664e-03,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
[제조]비유동자산회전률,9.711364e-04,-0.000000e+00,-0.000000e+00,-0.000000e+00,-0.000000e+00
[제조]유동자산구성비율,7.638032e-04,-0.000000e+00,-0.000000e+00,-0.000000e+00,-0.000000e+00
[제조]매출채권회전률,7.263543e-04,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
[제조]총자본사업이익률,3.477668e-04,-0.000000e+00,-0.000000e+00,-0.000000e+00,-0.000000e+00
[제조]자기자본구성비율,2.814893e-04,-0.000000e+00,-0.000000e+00,-0.000000e+00,-0.000000e+00
[제조]부채비율,1.728272e-04,8.067027e-05,5.189188e-05,0.000000e+00,0.000000e+00


In [ ]:
# ridge_alphas =[0,0.1,0.2,0.3,0.4,0.5]
# sort_column = 'alpha:'+str(ridge_alphas)
# coeff_df.sort_values(by=sort_column, ascending=False)

In [ ]:
# import matplotlib.pyplot as plt
# import numpy as np              
# from sklearn.metrics import mean_squared_error

# plt.scatter(Y_train,Y_pred_train, label = '(실제값, 예측값)', alpha=.5)
# plt.plot(np.linspace(0,20,200), np.linspace(0,50,200), color = 'green', label = "45°(실제값 = 예측값)")
# plt.xlabel("실제 Price: $Y_i$")
# plt.ylabel("예측 Price : $\hat{Y}_i$")
# plt.legend()
# plt.title("예측된 Price of train set vs 실제 Price ($Y_i$ vs $\hat{Y}_i$), " + f"alpha={alpha_value}")
# plt.text(y=0,x=35, s=f'MSE 값 : {mean_squared_error(Y_train,Y_pred_train) : .4f}')
# plt.show()

In [ ]:
# import matplotlib.pyplot as plt
# import numpy as np
# from sklearn.metrics import mean_squared_error

# plt.scatter(Y_test,Y_pred, label = '(실제값, 예측값)', alpha=1)
# plt.plot(np.linspace(0,50,200), np.linspace(0,50,200), color = 'green', label = "45°(실제값 = 예측값)")
# plt.xlabel("실제 Price: $Y_i$")
# plt.ylabel("예측 Price: $\hat{Y}_i$")
# plt.legend()
# plt.title("예측 Price of test set vs 실제 Price ($Y_i$ vs $\hat{Y}_i$), " + f"alpha={alpha_value}")
# plt.text(y=0,x=35, s=f'MSE 값 : {mean_squared_error(Y_test,Y_pred) : .4f}')
# plt.show()

In [ ]:
# # 잔차 분석
# resid = Y_pred_train - Y_train

# plt.scatter(Y_pred_train, resid, c='b')
# plt.hlines(y=0, xmin= -10, xmax=50)
# plt.title('Residual plot')
# plt.show()